In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("/Users/rachelp/Documents/OSHA_all_with_geocodes.csv")
df['city_and_state']= df.city+ ", " + df.state
df

In [ ]:
df["cause_details"]= df.date_of_incident + ", " + df.cause + "," + df.city_and_state
pd.options.display.max_colwidth = None
df.head()

In [ ]:
df1 = df.groupby(['state'])['state'].count().reset_index(name="count")
df1.tail(60)

In [ ]:
df1["string"] = df1["state"] + ": " + df1["count"].astype(str)
df1.tail(40)

In [ ]:
output = df.groupby('state')['cause_details'].apply(lambda x: "<div class='incident'><h1><b>Worker Fatalities:</b></h1><P>{0}</P></div>".format('</p><p> '.join(x))).reset_index(name='properties.article')
output

In [ ]:
deaths = df.groupby(['state'])['state'].count().reset_index(name="properties.headline")
deaths

In [96]:
deaths['properties.headline'] = deaths['state'] + " had " + deaths['properties.headline'].astype(str) + "workplace deaths between 2017 and September 2022."

In [97]:
output = output.merge(deaths, how='left', on= "state")

In [ ]:
ratings = df.groupby(['state'])['state'].count().reset_index(name="properties.rating")
ratings

In [ ]:
output = output.merge(ratings, how='left', on= "state")
output

In [115]:
import requests
import json
from pandas import json_normalize

In [129]:
with open('/Users/rachelp/Downloads/tl_2022_us_state (1) (1).json') as json_data:
    geometry_data = json.load(json_data)

In [130]:
df_geometry = pd.DataFrame.from_dict(json_normalize(geometry_data['features']), orient='columns')
df_geometry.head()

In [ ]:
df2 = df_geometry.drop(df_geometry.columns[[0,3, 4,5,6,7,10,11,12,13,14,15,16]], axis=1)
df2

In [133]:
final_df = output.merge(df2, how='left', left_on='state', right_on='properties.STUSPS')
final_df

In [135]:
ok_json = json.loads(final_df.to_json(orient='records'))
ok_json

In [137]:
def process_to_geojson(file):
    geo_data = {"type": "FeatureCollection", "features":[]}
    for row in file:
        this_dict = {"type": "Feature", "properties":{}, "geometry": {}}
        for key, value in row.items():
            key_names = key.split('.')
            if key_names[0] == 'geometry':
                this_dict['geometry'][key_names[1]] = value
            if str(key_names[0]) == 'properties':
                this_dict['properties'][key_names[1]] = value
        geo_data['features'].append(this_dict)
    return geo_data

In [138]:
geo_format = process_to_geojson(ok_json)
geo_format

In [140]:
with open('geo-data12-8.js', 'w') as outfile:
    outfile.write("var infoData = ")
#geojson output
with open('geo-data12-8.js', 'a') as outfile:
    json.dump(geo_format, outfile)